In [1]:
import pandas as pd
#import matplotlib.pyplot as plt
import numpy as np

### Load data

In [2]:
DatName='DYAD06NF__adjusted'
DatRaw=pd.read_excel('/Users/ruzenkakaldenbach/Desktop/Drive/raw_data_scaled/'+DatName+'.xlsx')
DatRaw

,Unnamed: 0,date,err,frame_count,frame_number,frame_timestamp,name,oid,type,x,y,x_cart,y_cart,x_cart_tr,y_cart_tr,x_tr,y_tr
0,0,2022-02-22T06:49:30.027330+01:00,NaN,0,0,0.000000,blue_center,1,BVIEW_annotated_object_2d,729.542549,513.611408,-230.457451,26.388592,-217.154639,18.682297,742.845361,521.317703
1,1,2022-02-22T06:49:30.027330+01:00,NaN,0,0,0.000000,blue_front,2,BVIEW_annotated_object_2d,743.034555,539.561709,-216.965445,0.438291,-204.491956,-4.689774,755.508044,544.689774
2,2,2022-02-22T06:49:30.027330+01:00,NaN,0,0,0.000000,blue_hat,3,BVIEW_annotated_object_2d,735.060775,526.115079,-224.939225,13.884921,-211.979518,7.397846,748.020482,532.602154
3,3,2022-02-22T06:49:30.027330+01:00,NaN,0,0,0.000000,red_center,5,BVIEW_annotated_object_2d,764.611729,880.072303,-195.388271,-340.072303,-184.860016,-315.016760,775.139984,855.016760
4,4,2022-02-22T06:49:30.027330+01:00,NaN,0,0,0.000000,red_front,6,BVIEW_annotated_object_2d,761.883141,874.760670,-198.116859,-334.760670,-187.420752,-310.232076,772.579248,850.232076
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308425,971,2022-02-22T08:26:51.360923+01:00,NaN,36148,36148,603.069133,red_front,6,BVIEW_annotated_object_2d,1051.591755,270.523982,91.591755,269.476018,86.884266,248.138342,1046.884266,291.861658
308426,972,2022-02-22T08:26:51.360923+01:00,NaN,36148,36148,603.069133,red_hat,7,BVIEW_annotated_object_2d,1051.443252,256.990785,91.443252,283.009215,86.772314,260.488560,1046.772314,279.511440
308427,973,2022-02-22T08:26:51.360923+01:00,NaN,36148,36148,603.069133,yellow_center,9,BVIEW_annotated_object_2d,1230.350601,885.085590,270.350601,-345.085590,254.094788,-308.667183,1214.094788,848.667183
308428,974,2022-02-22T08:26:51.360923+01:00,NaN,36148,36148,603.069133,yellow_front,10,BVIEW_annotated_object_2d,1214.486095,825.492977,254.486095,-285.492977,239.265629,-254.640575,1199.265629,794.640575


### Create an empty dataframe

Column name **xrc** referrs to **x**-coordinate of **r**ed hat **c**enter dot, etc.

In [3]:
DF = pd.DataFrame(columns=['xrc', 'yrc', 'xrf', 'yrf', 
                           'xbc', 'ybc', 'xbf', 'ybf', 
                           'xyc', 'yyc', 'xyf', 'yyf', 
                           'frame_timestamp'])
DF

,xrc,yrc,xrf,yrf,xbc,ybc,xbf,ybf,xyc,yyc,xyf,yyf,frame_timestamp


### Create an array

Based on the previous analysis, we have chosen a time interval of 0.25 seconds to divide the data, ensuring sufficient observations for each colour.

In [4]:
# Create an array with steps of 0,25 s until the max timestemp in DatRaw is reached
Seconds_025=np.arange(0,int(np.max(DatRaw['frame_timestamp'])),0.25)
Seconds_025

array([0.0000e+00, 2.5000e-01, 5.0000e-01, ..., 6.0225e+02, 6.0250e+02,
       6.0275e+02])

### Fill dataframe with values from DatRaw

In [5]:
# Map the color codes to their prefixes
color_map = {
    'r': 'red',
    'b': 'blue',
    'y': 'yellow'
}

for sec in Seconds_025:  # Loop through each interval
    # Filter data for the current time range
    dd = DatRaw[(DatRaw['frame_timestamp'] >= sec) & (DatRaw['frame_timestamp'] < sec + 0.25)]

    # Initialize a row for the current second
    row = {'frame_timestamp': sec}

    # Process each color
    for color_code, color_name in color_map.items():
        # Extract the data for center and front dots for the current color
        datcenter = dd[['x_tr', 'y_tr']][dd['name'].str.contains(f'{color_name}_center')]
        datfront = dd[['x_tr', 'y_tr']][dd['name'].str.contains(f'{color_name}_front')]

        if len(datcenter) > 0 and len(datfront) > 0:  # If both contain rows
            # Calculate the mean x_tr and y_tr coordinates for center and front dots
            row[f'x{color_code}c'] = np.mean(datcenter['x_tr'])
            row[f'y{color_code}c'] = np.mean(datcenter['y_tr'])
            row[f'x{color_code}f'] = np.mean(datfront['x_tr'])
            row[f'y{color_code}f'] = np.mean(datfront['y_tr'])

    # Append the row to the DataFrame
    DF = pd.concat([DF, pd.DataFrame([row])], ignore_index=True)

# Save to file
DF.to_csv("/Users/ruzenkakaldenbach/Desktop/filled.csv", index=False)

# Display the final DataFrame
DF

/var/folders/m7/mw8cknkx6b58s7sq4mdg8nhc0000gn/T/ipykernel_61959/3626359261.py:29: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  DF = pd.concat([DF, pd.DataFrame([row])], ignore_index=True)


,xrc,yrc,xrf,yrf,xbc,ybc,xbf,ybf,xyc,yyc,xyf,yyf,frame_timestamp
0,775.804288,854.928233,773.305632,850.077366,747.340300,503.109842,760.626050,526.102460,1003.121145,765.135096,984.325974,730.283877,0.00
1,774.659875,854.303370,772.400256,850.200750,732.087531,482.751365,752.993489,494.934314,1002.859481,765.117924,984.635502,729.859759,0.25
2,776.923568,853.740327,775.137394,848.614550,707.193442,468.437045,697.530720,470.975917,1002.528253,765.148044,984.580359,729.926126,0.50
3,777.604259,893.828964,777.506678,846.307486,674.576373,452.336553,659.247287,463.981262,1002.584897,765.855993,984.932742,730.450574,0.75
4,776.848484,899.206123,775.755358,844.106678,640.980919,450.287035,632.213531,470.021066,1002.720137,766.462569,984.809608,731.453564,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2407,1051.725138,272.946413,1049.134806,302.195526,584.354427,523.100070,610.415939,542.731424,1132.394116,839.061532,1125.831208,784.398438,601.75
2408,1049.742135,274.414157,1048.670725,300.908333,582.934081,491.469122,611.953540,527.955470,1130.843196,847.304774,1127.527744,792.628861,602.00
2409,1050.069746,274.338127,1048.713064,299.851005,NaN,NaN,NaN,NaN,1133.072429,844.052786,1128.692866,789.963874,602.25
2410,1049.310546,271.468227,1048.684367,296.539469,NaN,NaN,NaN,NaN,1155.644954,841.014203,1146.626374,787.364639,602.50


Reliability check: Compare number of means for each colour with Means_all_Intervals_all_Files:    
for DYAD06NF_ and	0.25s it should be   
blue 1911   
red  2124   
yellow  2290

In [6]:
# Iterate over colors and calculate the shape for each
for col in color_map.keys():  # Use the keys from color_map to iterate over 'r', 'b', 'y'
    # Filter rows for the current color
    color_df = DF[[f'x{col}c', f'y{col}c', f'x{col}f', f'y{col}f']].dropna(how='all')
    # Print the color and the shape of its subset
    print(color_map[col], color_df.shape)

red (2124, 4)
blue (1911, 4)
yellow (2290, 4)


### Interpolation

Replace missing values based on the value above and below the missing value within the same column.

In [7]:
from scipy.interpolate import interp1d

# Define a function to interpolate a column based on the row index (given the fix intervals between rows)
def interpolate_column_by_index(df, column_name):
    valid_idx = df.index[~df[column_name].isna()]  # Select row indices with valid (non-NaN) values
    valid_values = df[column_name][~df[column_name].isna()]  # Select the valid (non-NaN) values
    
    f = interp1d(valid_idx, valid_values, kind='linear', fill_value='extrapolate')
    df[column_name] = f(df.index)  # Interpolate for all row indices for the current column
    return df

# Apply interpolation for all coordinate columns
coordinate_columns = [col for col in DF.columns if col not in ['frame_timestamp']]
for column in coordinate_columns:
    DF = interpolate_column_by_index(DF, column)

# Save the interpolated DataFrame
DF.to_csv("/Users/ruzenkakaldenbach/Desktop/interpolated_by_index.csv", index=False)
DF

,xrc,yrc,xrf,yrf,xbc,ybc,xbf,ybf,xyc,yyc,xyf,yyf,frame_timestamp
0,775.804288,854.928233,773.305632,850.077366,747.340300,503.109842,760.626050,526.102460,1003.121145,765.135096,984.325974,730.283877,0.00
1,774.659875,854.303370,772.400256,850.200750,732.087531,482.751365,752.993489,494.934314,1002.859481,765.117924,984.635502,729.859759,0.25
2,776.923568,853.740327,775.137394,848.614550,707.193442,468.437045,697.530720,470.975917,1002.528253,765.148044,984.580359,729.926126,0.50
3,777.604259,893.828964,777.506678,846.307486,674.576373,452.336553,659.247287,463.981262,1002.584897,765.855993,984.932742,730.450574,0.75
4,776.848484,899.206123,775.755358,844.106678,640.980919,450.287035,632.213531,470.021066,1002.720137,766.462569,984.809608,731.453564,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2407,1051.725138,272.946413,1049.134806,302.195526,584.354427,523.100070,610.415939,542.731424,1132.394116,839.061532,1125.831208,784.398438,601.75
2408,1049.742135,274.414157,1048.670725,300.908333,582.934081,491.469122,611.953540,527.955470,1130.843196,847.304774,1127.527744,792.628861,602.00
2409,1050.069746,274.338127,1048.713064,299.851005,581.513735,459.838174,613.491142,513.179516,1133.072429,844.052786,1128.692866,789.963874,602.25
2410,1049.310546,271.468227,1048.684367,296.539469,580.093390,428.207227,615.028744,498.403562,1155.644954,841.014203,1146.626374,787.364639,602.50


### Add columns for distance between dyads

In [8]:
# based on squared distances of center dots
DF['dist_c_rb'] = (DF['xrc'] - DF['xbc'])**2 + (DF['yrc'] - DF['ybc'])**2
DF['dist_c_ry'] = (DF['xrc'] - DF['xyc'])**2 + (DF['yrc'] - DF['yyc'])**2
DF['dist_c_by'] = (DF['xbc'] - DF['xyc'])**2 + (DF['ybc'] - DF['yyc'])**2

# based on squared distances of front dots
DF['dist_f_rb'] = (DF['xrf'] - DF['xbf'])**2 + (DF['yrf'] - DF['ybf'])**2
DF['dist_f_ry'] = (DF['xrf'] - DF['xyf'])**2 + (DF['yrf'] - DF['yyf'])**2
DF['dist_f_by'] = (DF['xbf'] - DF['xyf'])**2 + (DF['ybf'] - DF['yyf'])**2

DF


,xrc,yrc,xrf,yrf,xbc,ybc,xbf,ybf,xyc,yyc,xyf,yyf,frame_timestamp,dist_c_rb,dist_c_ry,dist_c_by,dist_f_rb,dist_f_ry,dist_f_by
0,775.804288,854.928233,773.305632,850.077366,747.340300,503.109842,760.626050,526.102460,1003.121145,765.135096,984.325974,730.283877,0.00,124586.378888,59735.760653,134081.074254,105120.511877,58880.065051,91731.707528
1,774.659875,854.303370,772.400256,850.200750,732.087531,482.751365,752.993489,494.934314,1002.859481,765.117924,984.635502,729.859759,0.25,139863.296452,60029.103978,153048.322772,126590.863536,59525.754037,108847.987260
2,776.923568,853.740327,775.137394,848.614550,707.193442,468.437045,697.530720,470.975917,1002.528253,765.148044,984.580359,729.926126,0.50,153320.909717,58746.066326,175260.067842,148633.732747,57953.297577,149452.705627
3,777.604259,893.828964,777.506678,846.307486,674.576373,452.336553,659.247287,463.981262,1002.584897,765.855993,984.932742,730.450574,0.75,205530.293758,66993.368547,205884.031272,160158.624963,56448.396148,177076.909277
4,776.848484,899.206123,775.755358,844.106678,640.980919,450.287035,632.213531,470.021066,1002.720137,766.462569,984.809608,731.453564,1.00,219988.342591,68638.854316,230822.229616,160544.300683,56394.403712,192670.945014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2407,1051.725138,272.946413,1049.134806,302.195526,584.354427,523.100070,610.415939,542.731424,1132.394116,839.061532,1125.831208,784.398438,601.75,281012.234189,326993.812679,400179.147008,250331.763205,238401.986402,324055.845236
2408,1049.742135,274.414157,1048.670725,300.908333,582.934081,491.469122,611.953540,527.955470,1130.843196,847.304774,1127.527744,792.628861,602.00,265022.616876,334781.041543,426823.409947,242272.301363,248007.507747,335868.763873
2409,1050.069746,274.338127,1048.713064,299.851005,581.513735,459.838174,613.491142,513.179516,1133.072429,844.052786,1128.692866,789.963874,602.25,253955.003038,331464.237403,451837.859893,234927.174915,246607.393145,342042.396541
2410,1049.310546,271.468227,1048.684367,296.539469,580.093390,428.207227,615.028744,498.403562,1155.644954,841.014203,1146.626374,787.364639,602.50,244731.853939,335689.624955,501669.203273,228806.310961,250501.984068,366094.543751


### Add columns to indicate whether children are facing or "backing" each other

Based on the computed distances, identify whether the front dots or center dots are closer to each other.    
If front dots are closer than center dots, children are facing each other (value 1).   
If front dots are farther away than center dots, children are backing each other (value 0).  
optional: If the distance is approximately the same, children are parallel to each other (value 2). For now, just raise a warning.

In [11]:
import warnings

# Check for equal distances and raise warnings
if (DF['dist_c_ry'] == DF['dist_f_ry']).any():
    warnings.warn("Some rows have equal distances for central and front dots in Red-Yellow dyad (dist_c_ry == dist_f_ry).")

if (DF['dist_c_by'] == DF['dist_f_by']).any():
    warnings.warn("Some rows have equal distances for central and front dots in Blue-Yellow dyad (dist_c_by == dist_f_by).")

if (DF['dist_c_rb'] == DF['dist_f_rb']).any():
    warnings.warn("Some rows have equal distances for central and front dots in Red-Blue dyad (dist_c_rb == dist_f_rb).")

# Calculate the 'facing' columns
DF['facing_ry'] = np.where(DF['dist_c_ry'] > DF['dist_f_ry'], 1, 0)
DF['facing_by'] = np.where(DF['dist_c_by'] > DF['dist_f_by'], 1, 0)
DF['facing_rb'] = np.where(DF['dist_c_rb'] > DF['dist_f_rb'], 1, 0)

# Save to file
DF.to_csv("/Users/ruzenkakaldenbach/Desktop/columns.csv", index=False)

DF

,xrc,yrc,xrf,yrf,xbc,ybc,xbf,ybf,xyc,yyc,...,dist_f_by,vect_x_r,vect_y_r,vect_x_b,vect_y_b,vect_x_y,vect_y_y,facing_ry,facing_by,facing_rb
0,775.804288,854.928233,773.305632,850.077366,747.340300,503.109842,760.626050,526.102460,1003.121145,765.135096,...,91731.707528,-2.498656,-4.850866,13.285749,22.992618,-18.795170,-34.851219,1,1,1
1,774.659875,854.303370,772.400256,850.200750,732.087531,482.751365,752.993489,494.934314,1002.859481,765.117924,...,108847.987260,-2.259620,-4.102619,20.905958,12.182949,-18.223979,-35.258165,1,1,1
2,776.923568,853.740327,775.137394,848.614550,707.193442,468.437045,697.530720,470.975917,1002.528253,765.148044,...,149452.705627,-1.786174,-5.125777,-9.662722,2.538873,-17.947894,-35.221918,1,1,1
3,777.604259,893.828964,777.506678,846.307486,674.576373,452.336553,659.247287,463.981262,1002.584897,765.855993,...,177076.909277,-0.097581,-47.521477,-15.329085,11.644709,-17.652155,-35.405419,1,1,1
4,776.848484,899.206123,775.755358,844.106678,640.980919,450.287035,632.213531,470.021066,1002.720137,766.462569,...,192670.945014,-1.093127,-55.099445,-8.767389,19.734031,-17.910528,-35.009005,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2407,1051.725138,272.946413,1049.134806,302.195526,584.354427,523.100070,610.415939,542.731424,1132.394116,839.061532,...,324055.845236,-2.590332,29.249113,26.061512,19.631354,-6.562909,-54.663095,1,1,1
2408,1049.742135,274.414157,1048.670725,300.908333,582.934081,491.469122,611.953540,527.955470,1130.843196,847.304774,...,335868.763873,-1.071410,26.494176,29.019459,36.486348,-3.315452,-54.675913,1,1,1
2409,1050.069746,274.338127,1048.713064,299.851005,581.513735,459.838174,613.491142,513.179516,1133.072429,844.052786,...,342042.396541,-1.356682,25.512877,31.977407,53.341342,-4.379563,-54.088912,1,1,1
2410,1049.310546,271.468227,1048.684367,296.539469,580.093390,428.207227,615.028744,498.403562,1155.644954,841.014203,...,366094.543751,-0.626179,25.071242,34.935355,70.196336,-9.018580,-53.649564,1,1,1


### Add columns with vectoral head orientation for each child

In [12]:
# based on the center and front dots

DF['vect_x_r'] = DF['xrf'] - DF['xrc']  # Horizontal vector for red
DF['vect_y_r'] = DF['yrf'] - DF['yrc']  # Vertical vector for red

DF['vect_x_b'] = DF['xbf'] - DF['xbc']  # Horizontal vector for blue
DF['vect_y_b'] = DF['ybf'] - DF['ybc']  # Vertical vector for blue

DF['vect_x_y'] = DF['xyf'] - DF['xyc']  # Horizontal vector for yellow
DF['vect_y_y'] = DF['yyf'] - DF['yyc']  # Vertical vector for yellow

DF

,xrc,yrc,xrf,yrf,xbc,ybc,xbf,ybf,xyc,yyc,...,dist_f_by,vect_x_r,vect_y_r,vect_x_b,vect_y_b,vect_x_y,vect_y_y,facing_ry,facing_by,facing_rb
0,775.804288,854.928233,773.305632,850.077366,747.340300,503.109842,760.626050,526.102460,1003.121145,765.135096,...,91731.707528,-2.498656,-4.850866,13.285749,22.992618,-18.795170,-34.851219,1,1,1
1,774.659875,854.303370,772.400256,850.200750,732.087531,482.751365,752.993489,494.934314,1002.859481,765.117924,...,108847.987260,-2.259620,-4.102619,20.905958,12.182949,-18.223979,-35.258165,1,1,1
2,776.923568,853.740327,775.137394,848.614550,707.193442,468.437045,697.530720,470.975917,1002.528253,765.148044,...,149452.705627,-1.786174,-5.125777,-9.662722,2.538873,-17.947894,-35.221918,1,1,1
3,777.604259,893.828964,777.506678,846.307486,674.576373,452.336553,659.247287,463.981262,1002.584897,765.855993,...,177076.909277,-0.097581,-47.521477,-15.329085,11.644709,-17.652155,-35.405419,1,1,1
4,776.848484,899.206123,775.755358,844.106678,640.980919,450.287035,632.213531,470.021066,1002.720137,766.462569,...,192670.945014,-1.093127,-55.099445,-8.767389,19.734031,-17.910528,-35.009005,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2407,1051.725138,272.946413,1049.134806,302.195526,584.354427,523.100070,610.415939,542.731424,1132.394116,839.061532,...,324055.845236,-2.590332,29.249113,26.061512,19.631354,-6.562909,-54.663095,1,1,1
2408,1049.742135,274.414157,1048.670725,300.908333,582.934081,491.469122,611.953540,527.955470,1130.843196,847.304774,...,335868.763873,-1.071410,26.494176,29.019459,36.486348,-3.315452,-54.675913,1,1,1
2409,1050.069746,274.338127,1048.713064,299.851005,581.513735,459.838174,613.491142,513.179516,1133.072429,844.052786,...,342042.396541,-1.356682,25.512877,31.977407,53.341342,-4.379563,-54.088912,1,1,1
2410,1049.310546,271.468227,1048.684367,296.539469,580.093390,428.207227,615.028744,498.403562,1155.644954,841.014203,...,366094.543751,-0.626179,25.071242,34.935355,70.196336,-9.018580,-53.649564,1,1,1
